In [1]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.2/934.2 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.metrics import mape
from datetime import timedelta


In [18]:
import pandas as pd
data = pd.read_csv('/content/Daily_Public_Transport_Passenger_Journeys_by_Service_Type_20241128.csv')

In [19]:
data['Date'] = pd.to_datetime(data['Date'])

data.set_index('Date', inplace=True)

service_columns = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School']

time_series_dict = {}
for service in service_columns:
    time_series_dict[service] = TimeSeries.from_dataframe(data, value_cols=[service])

data.head()


<ipython-input-19-89ebc3249c94>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


,Local Route,Light Rail,Peak Service,Rapid Route,School,Other
Date,,,,,,
2024-08-30,16436,10705,225,19026,3925,59.0
2023-09-15,15499,10671,267,18421,4519,61.0
2021-12-28,1756,2352,0,3775,0,13.0
2023-01-11,10536,8347,223,14072,0,48.0
2021-09-11,820,612,0,1283,0,11.0


In [ ]:
forecast_horizon = 7
model_dict = {}
forecast_dict = {}

for service in service_columns:
    model = NBEATSModel(input_chunk_length=30, output_chunk_length=forecast_horizon, random_state=42)

    model.fit(time_series_dict[service])

    forecast = model.predict(n=forecast_horizon)

    model_dict[service] = model
    forecast_dict[service] = forecast


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 6.2 M  | train
-------------------------------------------------------------
6.2 M     Trainable params
1.5 K     Non-trainable params
6.2 M     Total params
24.972    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize=(14, 8))

for i, service in enumerate(service_columns, 1):
    plt.subplot(2, 3, i)
    time_series_dict[service].plot(label='Historical', color='blue')
    forecast_dict[service].plot(label=f'Forecast - {service}', color='red')

    plt.title(f'{service} Forecast (Next 7 Days)')
    plt.legend()

plt.tight_layout()
plt.show()


In [31]:

actual_values_dict = {}
for service in service_columns:
    mape_value = mape(actual_values_dict[service], forecast_dict[service])
    print(f'MAPE for {service}: {mape_value:.2f}')


**Prophet Model**

In [6]:
!pip install Prophet

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from datetime import timedelta


In [21]:
data

,Local Route,Light Rail,Peak Service,Rapid Route,School,Other
Date,,,,,,
2024-08-30,16436,10705,225,19026,3925,59.0
2023-09-15,15499,10671,267,18421,4519,61.0
2021-12-28,1756,2352,0,3775,0,13.0
2023-01-11,10536,8347,223,14072,0,48.0
2021-09-11,820,612,0,1283,0,11.0
...,...,...,...,...,...,...
2023-01-19,11337,8671,262,14881,0,66.0
2023-08-16,16791,11800,387,20936,4684,113.0
2022-01-10,5732,3548,73,6644,0,25.0


In [32]:
data['Date'] = pd.to_datetime(data['Date'])

service_columns = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School']

forecast_dict = {}

for service in service_columns:
    df_service = data[['Date', service]].rename(columns={'Date': 'ds', service: 'y'})

    model = Prophet(daily_seasonality=True, yearly_seasonality=False)

    model.fit(df_service)

    future = model.make_future_dataframe(df_service, periods=7)
    forecast = model.predict(future)

    forecast_dict[service] = forecast
